In [1]:
pip install ipytest

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importing required libraries
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pytest
import ipytest
ipytest.autoconfig()

In [3]:
dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv") #change path when testing

dataset_df = pd.DataFrame(dataset, columns=dataset.columns)
dataset_df["target"] = dataset_df["target"].astype('category')

In [9]:
class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Loading the dataset, and make the train, test, split."""
        dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv")
        
        # Assuming the target column is named 'Grade' and features are all other columns
        self.feature_names = dataset.columns.drop('Grade')
        self.frame = dataset
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame['Grade'], test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Execution method for running the pipeline several times."""
        self.load_dataset()
        self.train()


class PipelineWithFeatureEngineering(SimplePipeline):
    def __init__(self):
        super().__init__()
        self.scaler = StandardScaler()
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        self.load_dataset()
        self.apply_scaler()
        self.train()

In [10]:
pipeline = PipelineWithFeatureEngineering()
pipeline.run_pipeline()
accuracy_score = pipeline.get_accuracy()
print(f'The Accuracy of the model is: {accuracy_score}')

The Accuracy of the model is: 0.9157706093189965


## Testing

In [11]:
@pytest.fixture
def pipeline():
    pl = PipelineWithFeatureEngineering()
    pl.load_dataset()
    return pl

#### Creating the test

In [12]:
%%ipytest

def test_scaler_preprocessing_brings_x_train_mean_near_zero(pipeline):
    original_mean = pipeline.X_train.stack().mean()
    
    pipeline.apply_scaler()
    
    assert original_mean > pipeline.X_train.mean()
    assert np.isclose(pipeline.X_train.mean(), 0.0, atol=1e-3)

    print(f'The mean of the original X train is: {original_mean}')
    print(f'The mean of the transformed X train is: {pipeline.X_train.mean()}')

def test_scaler_preprocessing_brings_x_train_std_near_one(pipeline):
    pipeline.apply_scaler()
    
    assert np.isclose(pipeline.X_train.std(), 1.0, atol=1e-3)
    print(f'The SD of the transformed X train is : {pipeline.X_train.std()}')

..                                                                                           [100%]
2 passed in 0.14s


#### Making the test fails

In [13]:
%%ipytest

def test_scaler_preprocessing_brings_x_train_mean_near_zero(pipeline):
    original_mean = pipeline.X_train.stack().mean()
    
    pipeline.apply_scaler()

    # Changing the assertion, so it will fail
    assert original_mean < pipeline.X_train.mean()

    # Changing the value in isclose to make it fail
    assert not np.isclose(pipeline.X_train.mean(), 1.0, atol=1e-3)

F                                                                                            [100%]
============================================ FAILURES =============================================
_____________________ test_scaler_preprocessing_brings_x_train_mean_near_zero _____________________

pipeline = <__main__.PipelineWithFeatureEngineering object at 0x0000019D65E84850>

    def test_scaler_preprocessing_brings_x_train_mean_near_zero(pipeline):
        original_mean = pipeline.X_train.stack().mean()
    
        pipeline.apply_scaler()
    
        # Changing the assertion, so it will fail
>       assert original_mean < pipeline.X_train.mean()
E       assert 2.562661251791687 < -9.477297996481667e-18
E        +  where -9.477297996481667e-18 = <built-in method mean of numpy.ndarray object at 0x0000019D65E2AC10>()
E        +    where <built-in method mean of numpy.ndarray object at 0x0000019D65E2AC10> = array([[-1.22988009, -0.97262153, -0.25356194, ..., -0.16580533,\n        -

In [14]:
%%ipytest

def test_scaler_preprocessing_brings_x_train_std_near_one(pipeline):
    # Adding huge variation in the data
    pipeline.X_train *= 1000

    pipeline.apply_scaler()

    # Testing with the original tolerance
    assert np.isclose(pipeline.X_train.std(), 1.0, atol=1e-3)

F                                                                                            [100%]
============================================ FAILURES =============================================
______________________ test_scaler_preprocessing_brings_x_train_std_near_one ______________________

pipeline = <__main__.PipelineWithFeatureEngineering object at 0x0000019D65F1F0A0>

    def test_scaler_preprocessing_brings_x_train_std_near_one(pipeline):
        # Adding huge variation in the data
        pipeline.X_train *= 1000
    
        pipeline.apply_scaler()
    
        # Testing with the original tolerance
>       assert np.isclose(pipeline.X_train.std(), 1.0, atol=1e-3)
E       assert False
E        +  where False = <function isclose at 0x0000019D369F6C20>(1270.4740150219047, 1.0, atol=0.001)
E        +    where <function isclose at 0x0000019D369F6C20> = np.isclose
E        +    and   1270.4740150219047 = <built-in method std of numpy.ndarray object at 0x0000019D65EBAF70>()
E 